# IMT 574 Final Project: Weather Forecast

#Step 1: Data Import and Clean

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/IMT 574/DATASET/SeattleWeatherHistory.csv', sep=',')
df

,dt,dt_iso,timezone,city_name,lat,lon,temp,visibility,dew_point,feels_like,...,wind_gust,rain_1h,rain_3h,snow_1h,snow_3h,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,283996800,1979-01-01 00:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,22.96,NaN,7.57,15.57,...,NaN,NaN,NaN,NaN,NaN,99,804,Clouds,overcast clouds,04d
1,284000400,1979-01-01 01:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,23.18,NaN,7.36,15.04,...,NaN,NaN,NaN,NaN,NaN,100,804,Clouds,overcast clouds,04n
2,284004000,1979-01-01 02:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,22.95,NaN,7.14,15.35,...,NaN,NaN,NaN,NaN,NaN,53,803,Clouds,broken clouds,04n
3,284007600,1979-01-01 03:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,16.83,NaN,4.15,8.11,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
4,284011200,1979-01-01 04:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,16.63,NaN,3.96,7.93,...,NaN,NaN,NaN,NaN,NaN,0,800,Clear,sky is clear,01n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395731,1708628400,2024-02-22 19:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,53.37,NaN,46.35,52.02,...,10.00,NaN,NaN,NaN,NaN,99,804,Clouds,overcast clouds,04d
395732,1708632000,2024-02-22 20:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,54.28,NaN,46.18,52.90,...,8.99,0.25,NaN,NaN,NaN,70,500,Rain,light rain,10d
395733,1708635600,2024-02-22 21:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,55.85,NaN,46.60,54.46,...,10.00,0.25,NaN,NaN,NaN,70,500,Rain,light rain,10d
395734,1708639200,2024-02-22 22:00:00 +0000 UTC,-28800,Seattle,47.606139,-122.332848,56.12,NaN,46.11,54.68,...,5.99,NaN,NaN,NaN,NaN,70,803,Clouds,broken clouds,04d


In [ ]:
data = df[['dt_iso','temp','dew_point', 'feels_like', 'temp_min', 'temp_max',
       'pressure','humidity', 'wind_speed', 'wind_deg',
       'clouds_all','weather_main']]
data.head(5)

,dt_iso,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main
0,1979-01-01 00:00:00 +0000 UTC,22.96,7.57,15.57,20.82,24.67,1039,47,5.91,56,99,Clouds
1,1979-01-01 01:00:00 +0000 UTC,23.18,7.36,15.04,20.98,24.84,1039,46,6.78,63,100,Clouds
2,1979-01-01 02:00:00 +0000 UTC,22.95,7.14,15.35,20.48,24.89,1039,46,6.11,66,53,Clouds
3,1979-01-01 03:00:00 +0000 UTC,16.83,4.15,8.11,13.82,20.39,1039,53,6.04,72,0,Clear
4,1979-01-01 04:00:00 +0000 UTC,16.63,3.96,7.93,13.68,20.12,1039,53,5.99,79,0,Clear


In [ ]:
data = data.dropna(subset=['weather_main'])
data.isna().any()

dt_iso          False
temp            False
dew_point       False
feels_like      False
temp_min        False
temp_max        False
pressure        False
humidity        False
wind_speed      False
wind_deg        False
clouds_all      False
weather_main    False
dtype: bool

In [ ]:
print(data.dtypes)

dt_iso           object
temp            float64
dew_point       float64
feels_like      float64
temp_min        float64
temp_max        float64
pressure          int64
humidity          int64
wind_speed      float64
wind_deg          int64
clouds_all        int64
weather_main     object
dtype: object


In [ ]:
# Change dt_iso colunn to type datetime
data['dt_iso'] = pd.to_datetime(data['dt_iso'].apply(lambda x: x.split('+')[0]))

In [ ]:
# Extract date and time components
data['date'] = data['dt_iso'].dt.date
data['time'] = data['dt_iso'].dt.time
data

,dt_iso,temp,dew_point,feels_like,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_main,date,time
0,1979-01-01 00:00:00,22.96,7.57,15.57,20.82,24.67,1039,47,5.91,56,99,Clouds,1979-01-01,00:00:00
1,1979-01-01 01:00:00,23.18,7.36,15.04,20.98,24.84,1039,46,6.78,63,100,Clouds,1979-01-01,01:00:00
2,1979-01-01 02:00:00,22.95,7.14,15.35,20.48,24.89,1039,46,6.11,66,53,Clouds,1979-01-01,02:00:00
3,1979-01-01 03:00:00,16.83,4.15,8.11,13.82,20.39,1039,53,6.04,72,0,Clear,1979-01-01,03:00:00
4,1979-01-01 04:00:00,16.63,3.96,7.93,13.68,20.12,1039,53,5.99,79,0,Clear,1979-01-01,04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395731,2024-02-22 19:00:00,53.37,46.35,52.02,50.54,53.89,1027,77,8.99,194,99,Clouds,2024-02-22,19:00:00
395732,2024-02-22 20:00:00,54.28,46.18,52.90,52.05,55.74,1027,74,7.00,203,70,Rain,2024-02-22,20:00:00
395733,2024-02-22 21:00:00,55.85,46.60,54.46,52.05,56.75,1027,71,5.01,188,70,Rain,2024-02-22,21:00:00
395734,2024-02-22 22:00:00,56.12,46.11,54.68,54.05,56.75,1026,69,4.00,84,70,Clouds,2024-02-22,22:00:00


# Step 2: Data Transformation

In [ ]:
# Define the vectorized function to check if it rained between 8am and 8pm
def check_rain_vectorized(df):
    rain_hours_mask = (df['dt_iso'].dt.hour >= 8) & (df['dt_iso'].dt.hour <= 20) & (df['weather_main'].str.lower().str.contains('rain'))
    rain_dates = df[rain_hours_mask]['dt_iso'].dt.date.unique()

    # Create a boolean mask indicating whether it rained on each date
    rain_mask = df['dt_iso'].dt.date.isin(rain_dates)

    # Convert the boolean mask to 'Rain' and 'No Rain'
    return np.where(rain_mask, 'Rain', 'No Rain')

# Apply the vectorized function to create an array indicating if it rained between 8am and 8pm for all dates
rain_labels = check_rain_vectorized(data)

# Create a DataFrame with dates and rain labels
rain_df = pd.DataFrame({'date': data['dt_iso'].dt.date, 'rain_label': rain_labels})

# Group the rain labels by dates
grouped_rain = rain_df.groupby('date')['rain_label'].first()
grouped_rain.value_counts()

No Rain    8772
Rain       7717
Name: rain_label, dtype: int64

In [ ]:
# Get the features info (no labels here, labels calculated above already in vectorized function)
daily_data = data.groupby('date').agg({
    'feels_like': ['min', 'max'],
    'temp_min': 'min',
    'temp_max': 'max',
    'pressure': ['min', 'max', 'mean'],
    'humidity': ['min', 'max', 'mean'],
    'wind_speed': ['min', 'max', 'mean'],
    'wind_deg': ['min', 'max', 'mean'],
    'clouds_all': ['min', 'max', 'mean']
}).reset_index()

daily_data.columns = ['date', 'feels_like_min', 'feels_like_max', 'temp_min', 'temp_max',
                      'pressure_min', 'pressure_max', 'pressure_mean',
                      'humidity_min', 'humidity_max', 'humidity_mean',
                      'wind_speed_min', 'wind_speed_max', 'wind_speed_mean',
                      'wind_deg_min', 'wind_deg_max', 'wind_deg_mean',
                      'clouds_all_min', 'clouds_all_max', 'clouds_all_mean']
daily_data.set_index('date', inplace=True)
daily_data

,feels_like_min,feels_like_max,temp_min,temp_max,pressure_min,pressure_max,pressure_mean,humidity_min,humidity_max,humidity_mean,wind_speed_min,wind_speed_max,wind_speed_mean,wind_deg_min,wind_deg_max,wind_deg_mean,clouds_all_min,clouds_all_max,clouds_all_mean
date,,,,,,,,,,,,,,,,,,,
1979-01-01,6.12,18.61,12.04,26.11,1034,1041,1038.958333,44,60,53.208333,5.03,7.54,6.037917,56,85,76.583333,0,100,66.375000
1979-01-02,14.97,29.39,20.55,34.70,1027,1034,1031.000000,50,67,58.500000,3.71,5.95,4.921250,48,85,68.041667,72,100,94.000000
1979-01-03,18.86,30.56,23.79,37.20,1023,1027,1025.041667,52,69,62.750000,5.14,7.76,6.531250,33,80,64.208333,17,93,56.541667
1979-01-04,22.39,32.49,27.25,38.55,1023,1029,1025.791667,54,64,59.208333,6.58,8.21,7.360000,24,41,33.291667,28,100,80.708333
1979-01-05,18.57,30.87,22.10,38.50,1024,1031,1028.750000,50,63,58.166667,5.82,7.76,6.591667,31,54,39.958333,0,14,0.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-18,39.70,49.55,35.82,53.26,1008,1011,1009.416667,43,84,64.291667,1.01,5.01,2.527917,75,238,157.333333,95,100,99.375000
2024-02-19,39.38,47.89,40.89,48.88,1004,1009,1006.375000,78,94,86.916667,1.01,11.01,3.544167,0,355,135.291667,60,100,89.000000
2024-02-20,41.49,52.90,40.05,54.90,1006,1012,1009.458333,85,98,94.208333,1.01,5.99,2.562917,4,355,183.500000,70,100,95.875000


In [ ]:
# Combine labels
daily_data['weather_label'] = grouped_rain
daily_data.head(5)

,feels_like_min,feels_like_max,temp_min,temp_max,pressure_min,pressure_max,pressure_mean,humidity_min,humidity_max,humidity_mean,wind_speed_min,wind_speed_max,wind_speed_mean,wind_deg_min,wind_deg_max,wind_deg_mean,clouds_all_min,clouds_all_max,clouds_all_mean,weather_label
date,,,,,,,,,,,,,,,,,,,,
1979-01-01,6.12,18.61,12.04,26.11,1034,1041,1038.958333,44,60,53.208333,5.03,7.54,6.037917,56,85,76.583333,0,100,66.375000,No Rain
1979-01-02,14.97,29.39,20.55,34.70,1027,1034,1031.000000,50,67,58.500000,3.71,5.95,4.921250,48,85,68.041667,72,100,94.000000,No Rain
1979-01-03,18.86,30.56,23.79,37.20,1023,1027,1025.041667,52,69,62.750000,5.14,7.76,6.531250,33,80,64.208333,17,93,56.541667,No Rain
1979-01-04,22.39,32.49,27.25,38.55,1023,1029,1025.791667,54,64,59.208333,6.58,8.21,7.360000,24,41,33.291667,28,100,80.708333,No Rain
1979-01-05,18.57,30.87,22.10,38.50,1024,1031,1028.750000,50,63,58.166667,5.82,7.76,6.591667,31,54,39.958333,0,14,0.958333,No Rain


In [ ]:
print('Columns:', len(daily_data.columns))
print('Days:', len(daily_data))

Columns: 20
Days: 16489


### Make n-number of columns with day-n features

In [ ]:
## if change number n => run code from the top!
## BECAUSE USES PD.CONCAT, SO RUNNING JUST THIS BLOCK WILL MAKE daily_data DATAFRAME CONTINUE TO GO CRAZY BIG
n = 3

features = daily_data[['feels_like_min', 'feels_like_max', 'temp_min', 'temp_max',
                      'pressure_min', 'pressure_max', 'pressure_mean',
                      'humidity_min', 'humidity_max', 'humidity_mean',
                      'wind_speed_min', 'wind_speed_max', 'wind_speed_mean',
                      'wind_deg_min', 'wind_deg_max', 'wind_deg_mean',
                      'clouds_all_min', 'clouds_all_max', 'clouds_all_mean']]

# Shift the dataset to create features for the previous n days
for i in range(1, n + 1):
    shifted = features.shift(periods=i).add_suffix(f"_day_{i}")
    daily_data = pd.concat([daily_data, shifted], axis=1)

# Removing rows with NaN values that result from shifting
df_cleaned = daily_data.dropna()

# Get column names that contain '_day_' or 'weather_label'
selected_columns = [col for col in daily_data.columns if '_day_' in col or col == 'weather_label']

# Filter the DataFrame to include only selected columns
selected_data = daily_data[selected_columns]
selected_data

,weather_label,feels_like_min_day_1,feels_like_max_day_1,temp_min_day_1,temp_max_day_1,pressure_min_day_1,pressure_max_day_1,pressure_mean_day_1,humidity_min_day_1,humidity_max_day_1,...,humidity_mean_day_3,wind_speed_min_day_3,wind_speed_max_day_3,wind_speed_mean_day_3,wind_deg_min_day_3,wind_deg_max_day_3,wind_deg_mean_day_3,clouds_all_min_day_3,clouds_all_max_day_3,clouds_all_mean_day_3
date,,,,,,,,,,,,,,,,,,,,,
1979-01-01,No Rain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-02,No Rain,6.12,18.61,12.04,26.11,1034.0,1041.0,1038.958333,44.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-03,No Rain,14.97,29.39,20.55,34.70,1027.0,1034.0,1031.000000,50.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979-01-04,No Rain,18.86,30.56,23.79,37.20,1023.0,1027.0,1025.041667,52.0,69.0,...,53.208333,5.03,7.54,6.037917,56.0,85.0,76.583333,0.0,100.0,66.375000
1979-01-05,No Rain,22.39,32.49,27.25,38.55,1023.0,1029.0,1025.791667,54.0,64.0,...,58.500000,3.71,5.95,4.921250,48.0,85.0,68.041667,72.0,100.0,94.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-18,No Rain,36.09,49.86,34.05,53.91,1011.0,1022.0,1017.625000,42.0,76.0,...,83.500000,1.99,10.00,5.832500,0.0,315.0,94.958333,90.0,100.0,99.583333
2024-02-19,Rain,39.70,49.55,35.82,53.26,1008.0,1011.0,1009.416667,43.0,84.0,...,79.916667,1.01,8.99,3.454583,12.0,355.0,159.041667,5.0,100.0,48.041667
2024-02-20,Rain,39.38,47.89,40.89,48.88,1004.0,1009.0,1006.375000,78.0,94.0,...,57.250000,1.01,11.99,3.238333,23.0,355.0,212.125000,99.0,100.0,99.875000


In [ ]:
clean_df = selected_data.dropna()

In [ ]:
## Export as csv
clean_df.to_csv('/content/drive/MyDrive/cleaned_weather.csv', index=True)

# Step 3: Model Selection

### Train-test Split

In [ ]:
X = clean_df.drop(columns='weather_label')
y = clean_df['weather_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
clean_df

,weather_label,feels_like_min_day_1,feels_like_max_day_1,temp_min_day_1,temp_max_day_1,pressure_min_day_1,pressure_max_day_1,pressure_mean_day_1,humidity_min_day_1,humidity_max_day_1,...,humidity_mean_day_3,wind_speed_min_day_3,wind_speed_max_day_3,wind_speed_mean_day_3,wind_deg_min_day_3,wind_deg_max_day_3,wind_deg_mean_day_3,clouds_all_min_day_3,clouds_all_max_day_3,clouds_all_mean_day_3
date,,,,,,,,,,,,,,,,,,,,,
1979-01-04,No Rain,18.86,30.56,23.79,37.20,1023.0,1027.0,1025.041667,52.0,69.0,...,53.208333,5.03,7.54,6.037917,56.0,85.0,76.583333,0.0,100.0,66.375000
1979-01-05,No Rain,22.39,32.49,27.25,38.55,1023.0,1029.0,1025.791667,54.0,64.0,...,58.500000,3.71,5.95,4.921250,48.0,85.0,68.041667,72.0,100.0,94.000000
1979-01-06,No Rain,18.57,30.87,22.10,38.50,1024.0,1031.0,1028.750000,50.0,63.0,...,62.750000,5.14,7.76,6.531250,33.0,80.0,64.208333,17.0,93.0,56.541667
1979-01-07,No Rain,18.45,32.70,22.91,39.33,1021.0,1024.0,1022.166667,45.0,68.0,...,59.208333,6.58,8.21,7.360000,24.0,41.0,33.291667,28.0,100.0,80.708333
1979-01-08,No Rain,20.80,32.77,22.84,39.33,1022.0,1025.0,1023.625000,46.0,63.0,...,58.166667,5.82,7.76,6.591667,31.0,54.0,39.958333,0.0,14.0,0.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-18,No Rain,36.09,49.86,34.05,53.91,1011.0,1022.0,1017.625000,42.0,76.0,...,83.500000,1.99,10.00,5.832500,0.0,315.0,94.958333,90.0,100.0,99.583333
2024-02-19,Rain,39.70,49.55,35.82,53.26,1008.0,1011.0,1009.416667,43.0,84.0,...,79.916667,1.01,8.99,3.454583,12.0,355.0,159.041667,5.0,100.0,48.041667
2024-02-20,Rain,39.38,47.89,40.89,48.88,1004.0,1009.0,1006.375000,78.0,94.0,...,57.250000,1.01,11.99,3.238333,23.0,355.0,212.125000,99.0,100.0,99.875000


### Model 1: Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,f1_score
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier


In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
predictions = dt.predict(X_test)

print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.64941366761019
[[1797  822]
 [ 912 1415]]
              precision    recall  f1-score   support

     No Rain       0.66      0.69      0.67      2619
        Rain       0.63      0.61      0.62      2327

    accuracy                           0.65      4946
   macro avg       0.65      0.65      0.65      4946
weighted avg       0.65      0.65      0.65      4946



In [ ]:
precision = precision_score(y_test, predictions, pos_label='Rain')
recall = recall_score(y_test, predictions, pos_label='Rain')
f1 = f1_score(y_test, predictions, pos_label='Rain')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.6325435851586947
Recall: 0.6080790717662226
F1 Score: 0.6200701139351446


### Model 2: Random Forest

In [ ]:
rfc = RandomForestClassifier(n_estimators=200)
rfc.fit(X_train, y_train)
predictions = rfc.predict(X_test)
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

0.744237767893247
[[2010  609]
 [ 656 1671]]
              precision    recall  f1-score   support

     No Rain       0.75      0.77      0.76      2619
        Rain       0.73      0.72      0.73      2327

    accuracy                           0.74      4946
   macro avg       0.74      0.74      0.74      4946
weighted avg       0.74      0.74      0.74      4946



In [ ]:
precision = precision_score(y_test, predictions, pos_label='Rain')
recall = recall_score(y_test, predictions, pos_label='Rain')
f1 = f1_score(y_test, predictions, pos_label='Rain')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.7328947368421053
Recall: 0.7180919639020198
F1 Score: 0.7254178424137183


### Model 3: SVM

In [ ]:
# SVM
svm_model = SVC()
svm_model.fit(X_train, y_train)
svm_preds = svm_model.predict(X_test)
print(accuracy_score(y_test, svm_preds))
print(confusion_matrix(y_test, svm_preds))

0.7007682976142338
[[1883  736]
 [ 744 1583]]


In [ ]:
precision = precision_score(y_test, svm_preds, pos_label='Rain')
recall = recall_score(y_test, svm_preds, pos_label='Rain')
f1 = f1_score(y_test, predictions, pos_label='Rain')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.6826218197498922
Recall: 0.6802750322303395
F1 Score: 0.7324923879947802


### Model 4: Neural Network

In [ ]:
# NN Classification
# Scale the features
scaler = StandardScaler()
daily_df_scaled = scaler.fit_transform(clean_df.drop(columns=['weather_label']))

In [ ]:
X = daily_df_scaled
y = clean_df['weather_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
mlp = MLPClassifier (solver= "sgd", max_iter= 14, verbose= True, random_state= 1, learning_rate_init= 0.01, hidden_layer_sizes= (2000, 1000, 2))
mlp.fit( X_train, y_train)

Iteration 1, loss = 0.60797982
Iteration 2, loss = 0.56388464
Iteration 3, loss = 0.54768763
Iteration 4, loss = 0.53601101
Iteration 5, loss = 0.52648179
Iteration 6, loss = 0.51844412
Iteration 7, loss = 0.51133889
Iteration 8, loss = 0.50466145
Iteration 9, loss = 0.49631840
Iteration 10, loss = 0.49073190
Iteration 11, loss = 0.48173855
Iteration 12, loss = 0.47522029
Iteration 13, loss = 0.46596223
Iteration 14, loss = 0.45848935


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (14) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(hidden_layer_sizes=(2000, 1000, 2), learning_rate_init=0.01,
              max_iter=14, random_state=1, solver='sgd', verbose=True)

In [ ]:
yhat_train_mlp = mlp.predict(X_train)
yhat_test_mlp = mlp.predict(X_test)
print("Multylayer perceptron accuracy for training set is %.2f" % (accuracy_score(y_train, yhat_train_mlp)))
print("Multylayer perceptron accuracy for test set is %.4f" % (accuracy_score(y_test, yhat_test_mlp)))

Multylayer perceptron accuracy for training set is 0.80
Multylayer perceptron accuracy for test set is 0.7473


In [ ]:
precision = precision_score(y_test, yhat_test_mlp, pos_label='Rain')
recall = recall_score(y_test, yhat_test_mlp, pos_label='Rain')
f1 = f1_score(y_test, yhat_test_mlp, pos_label='Rain')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.7672456575682383
Recall: 0.6643747314138375
F1 Score: 0.7121142330723168


### Model 5: KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [ ]:
results = []

for k in range(1, 51, 2):
  classifier = KNeighborsClassifier(n_neighbors=k)
  classifier.fit(X_train,y_train)
  prediction = classifier.predict(X_test)
  correct = np.where(prediction==y_test, 1, 0).sum()
  accuracy = correct/len(y_test)
  print ("k=", k, " Accuracy=", accuracy)
  results.append([k,accuracy])

results = pd.DataFrame(results, columns=["k","accuracy"])

k= 1  Accuracy= 0.6389001213101496
k= 3  Accuracy= 0.6738778811160534
k= 5  Accuracy= 0.6892438334007278
k= 7  Accuracy= 0.6914678528103518
k= 9  Accuracy= 0.6971289931257582
k= 11  Accuracy= 0.7058228871815608
k= 13  Accuracy= 0.7086534573392641
k= 15  Accuracy= 0.7092600080873434
k= 17  Accuracy= 0.7145167812373635
k= 19  Accuracy= 0.7122927618277396
k= 21  Accuracy= 0.7149211484027497
k= 23  Accuracy= 0.7169429842296805
k= 25  Accuracy= 0.7169429842296805
k= 27  Accuracy= 0.7151233319854428
k= 29  Accuracy= 0.7185604528912253
k= 31  Accuracy= 0.7183582693085322
k= 33  Accuracy= 0.720380105135463
k= 35  Accuracy= 0.7232106752931662
k= 37  Accuracy= 0.7197735543873838
k= 39  Accuracy= 0.7189648200566114
k= 41  Accuracy= 0.7181560857258391
k= 43  Accuracy= 0.7185604528912253
k= 45  Accuracy= 0.7215932066316215
k= 47  Accuracy= 0.7193691872219976
k= 49  Accuracy= 0.7205822887181561


In [ ]:
## precision & recall for k = 49

precision = precision_score(y_test, prediction, pos_label='Rain')
recall = recall_score(y_test, prediction, pos_label='Rain')
f1 = f1_score(y_test, prediction, pos_label='Rain')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.7073277753400614
Recall: 0.6927374301675978
F1 Score: 0.6999565783760313


#Step 4: Weather Prediction (for presentation day Mar 7th)

In [ ]:
# Collecting data between March 4th and March 6th to forcast the weather of March 7th as a conlusion of our project
new_observed_data = [[
    43.63, 57.11, 42.06, 61.74, 1007.0, 1017.0, 1011.416667,
    28.0, 58.0, 41.333333, 1.01, 10.0, 4.733333, 10.0, 309.0,
    68.75, 0.0, 100.0, 87.25, 31.21, 54.57, 31.73, 58.89, 1018.0,
    1027.0, 1024.291667, 36.0, 92.0, 72.708333, 1.01, 11.99, 6.164167,
    6.0, 355.0, 211.666667, 13.0, 96.0, 49.666667, 34.72, 47.55,
    37.62, 50.68, 1023.0, 1026.0, 1024.5, 50.0, 87.0, 74.958333,
    1.99, 8.01, 4.835, 126.0, 279.0, 201.708333, 26.0, 100.0, 55.083333
]]

prediction_weekend_df = pd.DataFrame(new_observed_data, columns=
        ['feels_like_min_day_1', 'feels_like_max_day_1',
       'temp_min_day_1', 'temp_max_day_1', 'pressure_min_day_1',
       'pressure_max_day_1', 'pressure_mean_day_1', 'humidity_min_day_1',
       'humidity_max_day_1', 'humidity_mean_day_1', 'wind_speed_min_day_1',
       'wind_speed_max_day_1', 'wind_speed_mean_day_1', 'wind_deg_min_day_1',
       'wind_deg_max_day_1', 'wind_deg_mean_day_1', 'clouds_all_min_day_1',
       'clouds_all_max_day_1', 'clouds_all_mean_day_1', 'feels_like_min_day_2',
       'feels_like_max_day_2', 'temp_min_day_2', 'temp_max_day_2',
       'pressure_min_day_2', 'pressure_max_day_2', 'pressure_mean_day_2',
       'humidity_min_day_2', 'humidity_max_day_2', 'humidity_mean_day_2',
       'wind_speed_min_day_2', 'wind_speed_max_day_2', 'wind_speed_mean_day_2',
       'wind_deg_min_day_2', 'wind_deg_max_day_2', 'wind_deg_mean_day_2',
       'clouds_all_min_day_2', 'clouds_all_max_day_2', 'clouds_all_mean_day_2',
       'feels_like_min_day_3', 'feels_like_max_day_3', 'temp_min_day_3',
       'temp_max_day_3', 'pressure_min_day_3', 'pressure_max_day_3',
       'pressure_mean_day_3', 'humidity_min_day_3', 'humidity_max_day_3',
       'humidity_mean_day_3', 'wind_speed_min_day_3', 'wind_speed_max_day_3',
       'wind_speed_mean_day_3', 'wind_deg_min_day_3', 'wind_deg_max_day_3',
       'wind_deg_mean_day_3', 'clouds_all_min_day_3', 'clouds_all_max_day_3',
       'clouds_all_mean_day_3'])

prediction_weekend_df.to_csv("prediction_weekend_df.csv")

In [ ]:
predictions = rfc.predict(new_observed_data)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [ ]:
predictions

array(['No Rain'], dtype=object)